# TAR: Proyecto Final 

In [ ]:
%pip install gymnasium pandas numpy

## Tratamiento de datos

In [1]:
import numpy as np
import pandas as pd
import os

In [ ]:
ruta_datosMOP = "./Datos/MOP"
ruta_datosADME = "./Datos/ADME"
ruta_datos_procesados = "./Datos_procesados"

## Entorno personalizado para el problema hidro-térmico

In [ ]:
import gymnasium as gym
from gymnasium import spaces

class HydroThermalEnv(gym.Env):
    T_MAX = 52
    N_HIDRO = 5

    P_BON_MAX = 155
    P_BAY_MAX = 108
    P_PAL_MAX = 333
    P_SAL_MAX = 1890
    P_CLAIRE_MAX = P_BON_MAX + P_BAY_MAX + P_PAL_MAX + P_SAL_MAX

    Q_BON_MAX = 680
    Q_BAY_MAX = 828
    Q_PAL_MAX = 1372
    Q_SAL_MAX = 4200
    Q_CLAIRE_MAX = Q_BON_MAX + Q_BAY_MAX + Q_PAL_MAX + Q_SAL_MAX

    V_BON_MAX = 8200
    V_BAY_MAX = 0
    V_PAL_MAX = 1300
    V_SAL_MAX = 1500
    V_CLAIRE_MAX = V_BON_MAX + V_BAY_MAX + V_PAL_MAX + V_SAL_MAX

    K_CLAIRE = P_CLAIRE_MAX / Q_CLAIRE_MAX

    V0 = V_CLAIRE_MAX / 2

    def _inicial_hidrologia(self): ...

    def _siguiente_hidrologia(self, tiempo, hidrologia_actual): ...

    def _vertido(self, qt):
        return np.max(self.v - qt + self._aportes() - self.V_CLAIRE_MAX, 0)
    
    def _aportes(self): return 0
    
    def _demanda(self): return 0
    
    def _gen_eolico(self): return 0

    def _gen_solar(self): return 0

    def _gen_bio(self): return 0

    def _gen_termico_barato(self, demanda_residual): return 0

    def _gen_termico_caro(self, demanda_residual): return 0

    def _despachar(self, qt):
        demanda_residual = self._demanda() - self._gen_eolico() - self._gen_solar() - self._gen_bio() - (self.K_CLAIRE * qt)
        energia_termico_barato = 0
        energia_termico_caro = 0
        exportacion = 0

        if demanda_residual > 0:
            # Primero uso termico barato
            energia_termico_barato = self._gen_termico_barato(demanda_residual)
            demanda_residual -= energia_termico_barato

            if demanda_residual > 0:
                energia_termico_caro = self._gen_termico_caro(demanda_residual)
                demanda_residual -= energia_termico_caro

        if demanda_residual < 0:
            exportacion = -demanda_residual

        # Ajustar valores de ingreso por exportacion y costos de térmicos
        return exportacion * 50, energia_termico_barato * 100 + energia_termico_caro * 200

    def __init__(self):
        # Defino espacios de observación y acción
        self.observation_space = spaces.Dict({
            "volumen": spaces.Box(0.0, self.V_CLAIRE_MAX, shape=()),
            "hidrologia": spaces.Discrete(self.N_HIDRO),
            "tiempo": spaces.Discrete(52)
        })
        # turbinar en [0, V_MAX] continuo
        self.action_space = spaces.Box(0.0, self.V_CLAIRE_MAX, shape=())

    def reset(self, *, seed=None, options=None):
        self.v = self.V0
        self.t = 0
        self.h = self._inicial_hidrologia()
        info = {
            "volumen_inicial": self.v,
            "hidrologia_inicial": self.h,
            "tiempo_inicial": self.t
        }
        return self._get_obs(), info
    
    def step(self, action):
        # Validar que la acción esté en el espacio válido
        assert self.action_space.contains(action), f"Acción inválida: {action}. Debe estar en {self.action_space}"
        qt = float(action)
        
        # dinámica: v ← v − q − d + a
        self.v = self.v - qt - self._vertido(qt) + self._aportes()
        self.h = self._siguiente_hidrologia(self.t, self.h)
        self.t += 1

        # despacho: e_eolo + e_sol + e_bio + e_termico + e_hidro = dem + exp
        ingreso_exportacion, costo_termico = self._despachar(qt)

        # recompensa: −costo_termico + ingreso_exportacion
        reward = -costo_termico + ingreso_exportacion
        
        done = (self.t >= self.T_MAX)
        info = {
            "volumen": self.v,
            "tiempo": self.t,
            "hidrologia": self.h,
            "turbinado": qt,
            "vertido": self._vertido(qt),
            "ingreso_exportacion": ingreso_exportacion,
            "costo_termico": costo_termico,
        }
        return self._get_obs(), reward, done, False, info
    
    def render(self, mode='human'):
        if mode == 'human':
            print(f"Semana {self.t}/52:")
            print(f"  Volumen embalse: {self.v:.2f}/{self.V_CLAIRE_MAX}")
            print(f"  Estado hidrológico: {self.h}")
            print(f"  Porcentaje llenado: {(self.v/self.V_CLAIRE_MAX)*100:.1f}%")
            print("-" * 30)
        elif mode == 'rgb_array':
            # Retornar una imagen como array numpy para grabación
            pass
        elif mode == 'ansi':
            # Retornar string para mostrar en terminal
            return f"T:{self.t} V:{self.v:.1f} H:{self.h}"
        
    def _get_obs(self):
        # Mapeo de variables internas a observación del agente
        obs = {
            "volumen": self.v, 
            "hidrologia": self.h, 
            "tiempo": self.t
        }
        # Validar contra observation_space (opcional, útil para debug)
        assert self.observation_space.contains(obs), f"Observación inválida: {obs}. Debe estar en {self.observation_space}"
        return obs